# Разработка системы распознавания предметов интерьера в потоковом видео

## Выявление частоты классов в датасете

In [1]:
import os

import numpy as np
import pandas as pd

import seaborn as sns
sns.set(style="white")

%matplotlib inline
import matplotlib.pyplot as plt 

from skimage.io import imshow, imread

### Ранее написанные функции

 Загрузим ранее написанные функции для удобной работы с именами файлов и преобразования файлов с описанием в pandas.DataFrame

In [2]:
img_dir = 'data/ADE20K_filtred/images/'

In [3]:
def get_format(file_name):
    name_split = file_name.split("_")
    file_format = name_split[0:-1] + name_split[-1].split(".")
    ade, train_or_val, name = file_format[:3]
    extension = file_format[-1]
    
    description = 0
    parts_num = 0
    if extension == 'jpg':
        description = 'img'
    elif extension == 'txt':
        description = 'text'
    else: #png
        description = file_format[3]
        if file_format[4].isdigit():
            parts_num = int(file_format[4])
    return [name, description, train_or_val, parts_num]

Но преобразуем их с учётом того, что мы будем работать только с верхними уровнями объектов

In [4]:
def get_seg_description(path):
    description = []
    with open(path) as f:
        for line in f:
            (instance_n, part_level, occluded,
            class_name, original_name, attributes_list) = line.rstrip().split(" # ")
            
            if part_level != '0':  #добавляем только элементы верхнего уровня
                continue
            
            description.append([class_name, original_name, attributes_list.strip("\"")])
    description = pd.DataFrame(description).rename(columns={0: "class_name", 1: "original_name", 2: "attributes" })
    return description

In [5]:
get_seg_description('data/ADE20K_filtred/images/train/a/art_gallery/ADE_val_00001033_atr.txt').head()

,class_name,original_name,attributes
0,wall,wall,
1,wall,wall,
2,wall,wall,
3,"floor, flooring",floor,
4,ceiling,ceiling,


### Получение полного списка классов

Для определения того, какие классы нам нужно детектировать выделим все классы из описаний фотографий.

In [6]:
def get_full_df_description(dir_path):
    df = pd.DataFrame(columns=["class_name", "original_name", "attributes"])
    s = 0
    for path, dirs, files in os.walk(dir_path):
        if files:
            for file in files:
                name, description, train_or_val, parts_num = get_format(file)
                if description == 'text':
                    s += 1
                    if s % 100 == 0:
                        print(str(s) + " done, size: " + str(df.shape) )
                        
                    df = df.append(get_seg_description(path +'/' + file), ignore_index=True)
    return df

In [7]:
%%time
full_df = get_full_df_description(img_dir)

100 done, size: (2052, 3)
200 done, size: (4253, 3)
300 done, size: (6221, 3)
400 done, size: (9244, 3)
500 done, size: (12506, 3)
600 done, size: (15634, 3)
700 done, size: (18910, 3)
800 done, size: (22271, 3)
900 done, size: (25436, 3)
1000 done, size: (28738, 3)
1100 done, size: (31426, 3)
1200 done, size: (33365, 3)
1300 done, size: (35641, 3)
1400 done, size: (37937, 3)
1500 done, size: (40692, 3)
1600 done, size: (43179, 3)
1700 done, size: (45356, 3)
1800 done, size: (48573, 3)
1900 done, size: (50809, 3)
2000 done, size: (53123, 3)
2100 done, size: (56188, 3)
2200 done, size: (59337, 3)
2300 done, size: (62153, 3)
2400 done, size: (65176, 3)
2500 done, size: (67396, 3)
2600 done, size: (69580, 3)
2700 done, size: (72029, 3)
2800 done, size: (73734, 3)
2900 done, size: (75407, 3)
3000 done, size: (77383, 3)
3100 done, size: (80116, 3)
3200 done, size: (81907, 3)
3300 done, size: (83909, 3)
3400 done, size: (85745, 3)
3500 done, size: (87642, 3)
3600 done, size: (90643, 3)
3700 

Посмотрим на получившиеся значения

In [8]:
full_df

,class_name,original_name,attributes
0,sky,sky,
1,wall,wall,
2,wall,wall,
3,wall,wall,
4,wall,wall,
...,...,...,...
240268,wall,wall,
240269,wall,wall,
240270,"earth, ground",ground,
240271,"animal, animate being, beast, brute, creature,...",animal,gorilla


Посмотрим на самые частые

In [9]:
full_df.class_name.value_counts()[:15]

wall                                                   25813
chair                                                  12476
floor, flooring                                         9012
painting, picture                                       8164
light, light source                                     8127
windowpane, window                                      7663
cabinet                                                 7272
table                                                   7229
ceiling                                                 6805
person, individual, someone, somebody, mortal, soul     6725
lamp                                                    5556
book                                                    4998
cushion                                                 4846
curtain, drape, drapery, mantle, pall                   4296
plant, flora, plant life                                3918
Name: class_name, dtype: int64

Посмотрим количество уникальных значений

In [10]:
full_df.class_name.unique().shape

(1879,)

Определим минимальное количество слов

In [40]:
min_obj = 48 #Минимальное количество объектов для равпознавания
indexes = list(full_df.class_name.value_counts() > min_obj)
full_df.class_name.value_counts()[indexes]
class_list = list(full_df.class_name.value_counts()[indexes].index)
len(class_list)

255

Удалим те, которые нам точно не нужны

In [41]:
exclusion_list = ['wall', 'floor', 'ceiling']

In [42]:
def remove_exclusion(class_list, exclusion_list):
    for val in exclusion_list:
        for class_ in class_list:
            class_arr = class_.split(', ')
            for class_inst in class_arr:
                if val == class_inst:
                    class_list.remove(class_)

In [43]:
remove_exclusion(class_list, exclusion_list)

И добавим "пустой" класс

In [44]:
class_list = ["-"] + class_list
class_list

['-',
 'chair',
 'painting, picture',
 'light, light source',
 'windowpane, window',
 'cabinet',
 'table',
 'person, individual, someone, somebody, mortal, soul',
 'lamp',
 'book',
 'cushion',
 'curtain, drape, drapery, mantle, pall',
 'plant, flora, plant life',
 'box',
 'door',
 'tree',
 'bottle',
 'building, edifice',
 'bed',
 'pillow',
 'armchair',
 'shelf',
 'vase',
 'flower',
 'sofa, couch, lounge',
 'glass, drinking glass',
 'plate',
 'sconce',
 'wall socket, wall plug, electric outlet, electrical outlet, outlet, electric receptacle',
 'mirror',
 'sink',
 'rug, carpet, carpeting',
 'pot, flowerpot',
 'spotlight, spot',
 'desk',
 'work surface',
 'sky',
 'towel',
 'column, pillar',
 'swivel chair',
 'basket, handbasket',
 'plaything, toy',
 'car, auto, automobile, machine, motorcar',
 'pot',
 'jar',
 'signboard, sign',
 'coffee table, cocktail table',
 'bowl',
 'stool',
 'seat',
 'candlestick, candle holder',
 'shoe',
 'chandelier, pendant, pendent',
 'clock',
 'railing, rail',
 

Мы получили список допустимых для нас классов

### Совмещение с индексами классов

In [45]:
obj_names = 0
with open('data/ADE20K_filtred/objectnames.txt', 'r') as f:
    obj_names = f.readline().split("\t")
    obj_names = list(map(lambda name: name.strip("\'"), obj_names))

In [46]:
obj_names.index('wall')

2977

In [47]:
class_index_dict = dict()
index_old_to_new = dict()
for class_name, i in zip(class_list, range(len(class_list))):
    class_index_dict[class_name] = obj_names.index(class_name)
    index_old_to_new[obj_names.index(class_name)] = i

In [48]:
class_index_dict.values()

dict_values([0, 470, 1734, 1450, 3054, 349, 2683, 1830, 1394, 235, 688, 686, 1909, 265, 773, 2854, 248, 311, 164, 1868, 56, 2328, 2931, 977, 2472, 1097, 1918, 2242, 2981, 1563, 2387, 2177, 1980, 2508, 723, 3086, 2419, 2820, 580, 2678, 145, 1929, 400, 1973, 1348, 2379, 570, 258, 2585, 2271, 377, 2340, 479, 529, 2052, 2675, 211, 2732, 490, 94, 1755, 1124, 981, 917, 1032, 2597, 2849, 238, 590, 2130, 2615, 2729, 942, 1484, 136, 2376, 1643, 2095, 180, 2792, 893, 370, 154, 1609, 1701, 2137, 2984, 37, 1968, 1582, 782, 626, 628, 1618, 1550, 119, 2530, 1001, 837, 2261, 777, 1943, 3108, 302, 63, 1196, 1259, 906, 676, 375, 1473, 1935, 2529, 746, 205, 1707, 1947, 1891, 1895, 241, 1743, 222, 896, 2568, 95, 761, 1157, 1345, 2045, 2338, 1661, 568, 1377, 219, 2722, 790, 122, 2988, 2263, 2954, 160, 2791, 28, 1809, 317, 1406, 2250, 419, 2464, 2721, 2747, 1138, 46, 2993, 953, 674, 1373, 195, 2355, 2098, 102, 2035, 1222, 2442, 1783, 2940, 1746, 1883, 1787, 2147, 244, 1185, 9, 307, 1985, 1366, 745, 384, 28

Мы получили отображение имени класса в его номер

In [49]:
index_old_to_new.items()

dict_items([(0, 0), (470, 1), (1734, 2), (1450, 3), (3054, 4), (349, 5), (2683, 6), (1830, 7), (1394, 8), (235, 9), (688, 10), (686, 11), (1909, 12), (265, 13), (773, 14), (2854, 15), (248, 16), (311, 17), (164, 18), (1868, 19), (56, 20), (2328, 21), (2931, 22), (977, 23), (2472, 24), (1097, 25), (1918, 26), (2242, 27), (2981, 28), (1563, 29), (2387, 30), (2177, 31), (1980, 32), (2508, 33), (723, 34), (3086, 35), (2419, 36), (2820, 37), (580, 38), (2678, 39), (145, 40), (1929, 41), (400, 42), (1973, 43), (1348, 44), (2379, 45), (570, 46), (258, 47), (2585, 48), (2271, 49), (377, 50), (2340, 51), (479, 52), (529, 53), (2052, 54), (2675, 55), (211, 56), (2732, 57), (490, 58), (94, 59), (1755, 60), (1124, 61), (981, 62), (917, 63), (1032, 64), (2597, 65), (2849, 66), (238, 67), (590, 68), (2130, 69), (2615, 70), (2729, 71), (942, 72), (1484, 73), (136, 74), (2376, 75), (1643, 76), (2095, 77), (180, 78), (2792, 79), (893, 80), (370, 81), (154, 82), (1609, 83), (1701, 84), (2137, 85), (2984

И отображение из старой кодировки в новую

In [50]:
import csv

with open('data/ADE20K_filtred/index_old_to_new.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(index_old_to_new.items())

In [51]:
with open('data/ADE20K_filtred/class_index_dict.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(class_index_dict.items())

In [250]:
with open('data/classes.txt', 'w') as f:
    for cl in class_list:
        f.write(cl + '\n')


In [269]:
filt_class_list = []
with open('data/filt_classes.txt', 'r') as f:
    for line in f:
        filt_class_list.append(line.strip())
filt_class_list = ["-"] + filt_class_list

In [275]:
class_index_dict = dict()
index_old_to_new = dict()
for class_name, i in zip(filt_class_list, range(len(filt_class_list))):
    index = -1
    
    for i, obj in zip(range(len(obj_names)), obj_names):
        obj_arr =  obj.split(',')
        for val in obj_arr:
            if val.strip() == class_name:
                index = i
    
    if index == -1:
        print('Error')
        break
        
    class_index_dict[class_name] = index
    index_old_to_new[index] = i

Error


In [268]:
'dustbin' in class_list

False

In [271]:
" a ".strip()

'a'

In [277]:
for i, obj in zip(range(len(obj_names)), obj_names):
        obj_arr =  obj.split(',')
        for val in obj_arr:
            print(val.strip())

-
aarm panel
abacus
accordion
piano accordion
squeeze box
acropolis
ad
advertisement
advertizement
advertising
advertizing
advert
adding machine
advertisement board
aerial
air conditioner
air conditioning
air hockey table
air machine
aircraft carrier
airplane
aeroplane
plane
airport cart
alarm
alarm clock
alembic
alga
algae
altar
communion table
Lord''s table
altarpiece
amphitheater
amphora
anchor
andiron
andirons
animal toy
animal
animate being
beast
brute
creature
fauna
animals
antenna
antenna
aerial
transmitting aerial
antler
antlers
anvil
aperture
apparatus
apparel
wearing apparel
dress
clothes
apple
apples
appliance
apron
aquarium
aqueduct
arbor
arcade
arcade machine
arcade machines
arcade
colonnade
arcades
arch
arch
archway
arches
arm
arm panel
arm support
armchair
armor
armrest
art
art mannequin
articulated lamp
artificial golf green
ashcan
trash can
garbage can
wastebin
ash bin
ash-bin
ashbin
dustbin
trash barrel
trash bin
ashtray
asymmetric bars
athletic field
athletics track


vessel
vest
viaduct
video
video camera
video game
video-tape
videocassette recorder
VCR
videos
viewpoint
village
vine
vine shoot
vinegar jar
vinery
vineyard
vintage bottle filler
violin
violoncello
visit cards
visor
volcano
volleyball court
voting booth
wafers
wagon
wagons
walker
walkie talkie
walking stick
walkway
wall
wall clock
wall mount
wall recessed light
wall socket
wall plug
electric outlet
electrical outlet
outlet
electric receptacle
wallet
walrus
wardrobe
closet
press
warehouse
warship
washbasin chair
washer
automatic washer
washing machine
washing hair
washing machines
watch
watchtower
water
water bog
water chute
water cooler
water dispenser
water ditch
water faucet
water tap
tap
hydrant
water filter
water hole
water lily
water machine
water mill
water mist
water pump
water spurt
water surf
water tower
water valve
water vapor
water wheel
watercraft
waterfall
waterfall
falls
watering can
waterscape
waterway
wave
waves
way
weapon
weapons
weathervane
webcam
weeds
weighbridge
we

In [278]:
filt_class_list

['-',
 'chair',
 'picture',
 'cabinet',
 'table',
 'person',
 'lamp',
 'plant',
 'door',
 'bed',
 'armchair',
 'flower',
 'sofa',
 'sink',
 'spotlight',
 'desk',
 'signboard, sign',
 'coffee table',
 'stool',
 'seat',
 'chandelier',
 'clock',
 'switch',
 'tv',
 'chest',
 'stove',
 'computer',
 'bookcase',
 'telephone',
 'fireplace',
 'refrigerator',
 'bathtub',
 'wardrobe',
 'monitor',
 'microwave',
 'stairway',
 'screen',
 'trash bin',
 'speaker',
 'stairs',
 'dishwasher',
 'oven',
 'pool table',
 'coffee maker',
 'board',
 'teapot',
 'washer',
 'bulletin board',
 'laptop',
 'grill',
 'shower',
 'remote',
 'air conditioner',
 'boiler',
 'screen',
 'console',
 'printer',
 'cradle',
 'system',
 'microphone',
 'keyboard',
 'toaster',
 'ladder',
 'shower stall',
 'blackboard',
 'piano',
 'vending machine',
 'fire extinguisher',
 'projector',
 'table football',
 'rocking chair',
 'hot tub']

In [283]:
for
for val in obj_names:
    if val.find('sofa') != -1:
        print(val)

miniature sofa
sofa bed
sofa, couch, lounge


In [281]:
"dsfdssofa, asdf".find('sofa')

5